# Recommending U.S. Cities for Coffee Equipment Supplier
Ken Takekura<br>
9 Jul 2020

## Introduction
The purpose of this report is to give advice to a hypothetical client, a successful commercial coffee-making equipment supplier based in Seattle, U.S.<br>
The client has supplied items such as coffee machines, grinders and paper cups, to a number of local coffee shops in Seattle. After big success in Seattle, they are now looking to expand their business into other cities in U.S. They are seeking advice which city to choose that gives the best possible chance of success.
In order to recommend target market, I decided to choose 3 candidate cities that meet all following criteria:<br><br>
**1. Largely populated**<br>
**2. Structure of the city (i.e. what kind of places are around) is similar to Seattle.**<br>
**3. There are sufficient number of coffee shops to serve.**<br><br>
In this analysis, I used Foursquare as the main source of data, as it provides detailed information of venues located in each city, such as venue category and geo coordinates, which will be used to meet criteria 2 and 3. I also used a Wikipedia page of most populous cities in U.S. to meet criteria 1.

## Data Source
In this analysis, I used below data sources:
- <ins>'List of United States cities by population' from Wikipedia</ins><br>
(https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population)<br>
This data source was used to get a list of highly populated cities in US.<br>
Applicable data frame was scraped from the html script, and only top 20 rows were used as the first screening process.<br>
I used only two columns: 'City' and 'State' (e.g. 'City': Chicago, 'State': Illinois) for the sake of simplicity.<br><br>
- <ins>'Nominatim OpenStreetMap Data'</ins><br>
The wikipedia dataframe actually comes with 'Location' column which shows geographical coordinates of the cities, however I did not use this data because these locations merely show the central point when the whole area of the city is shown on a map, not necessarily pointing the central business district of the city which is more relevant for this analysis.<br>
Instead, I retrieved geo coordinates from Nominatim’s map data using GeoPy library, as it points the location of the municipal government’s office (e.g. city hall), which is in most cases located within the central business district.<br><br>
- <ins>Foursquare Places Data</ins><br>
I used Foursquare's 'explore' API call to get local venues information including latitude, longitude, and venue category. (e.g. latitude:40.71365285, longitude: 74.0088039, category: Bakery)<br>
Obtained data were used to cluster the cities. For example, Seattle and New York will be in the same cluster as these cities share a number of common venues such as coffee shops and pizza places.<br>
Additionally, the number of coffee shops in each city was used as the basis of choosing final 3 candidate cities.

## Methodology
Below steps were taken to reach final 3 candidate cities:<br><br>
**1. Meet criteria #1: Largely populated**<br>
Scrape data frame from wikipedia page 'List of United States cities by population'.<br>
Once data frame is captured, only first 20 are used, meaning top 20 populous cities are selected at this stage.<br><br>
**2. Meet criteria #2: Similar structure to Seattle**<br>
In this analysis, ‘similar structure’ of the city is defined as sharing similar venues such as bars, restaurants, parks, museums, and so on.<br>
This similarity of cities is found by getting venue information of each city from Foursquare's Places Data and clustering the cities based on their local venues.<br>
K-means clustering method is used to group the cities into clusters and the number of clusters is set as 5 (i.e. k=5).<br>
Of the 20 cities selected in Step 1, those in the same cluster as Seattle are screened.<br><br>
**3. Meet criteria #3: Sufficient number of coffee shops**<br>
Of those cities screened in the previous step, 3 final candidate cities are selected based on the number of possible target coffee shops.<br>
Here again, Foursquare's data in Step 2 is used to identify how many coffee shops are in each city.

First import necessary packages.

In [10]:
import re
import pandas as pd
!conda install lxml --yes
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
import numpy as np
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be UPDATED:

  ca-certificates    conda-forge::ca-certificates-2020.6.2~ --> pkgs/main::ca-certificates-2020.6.24-0

The following packages will be SUPERSEDED by a higher-priority channel:

  certifi            conda-forge::certifi-2020.6.20-py36h9~ --> pkgs/main::certifi-2020.6.20-py36_0
  openssl            conda-forge::openssl-1.1.1g-h516909a_0 --> pkgs/main::openssl-1.1.1g-h7b6447c_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    pkgs/main::ca-certificates-2020.6.24-0 --> conda-forge::ca-certificates-2020.6.20-hecda079_0
  certifi        

Scrape html codes from wikipedia page and retrieve the dataframe we need.<br>
There were multiple data frames within the html script, and the applicable one was the 4th data frame.

In [11]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population')
df_cities = pd.DataFrame(df[4].iloc[:20, 1:3])
for i, val in enumerate(df_cities['City']):
    df_cities.loc[i, 'City'] = re.sub(r"\[[\w]]", "", val) 
df_cities.head(20)

,City,State[c]
0,New York,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Phoenix,Arizona
5,Philadelphia,Pennsylvania
6,San Antonio,Texas
7,San Diego,California
8,Dallas,Texas
9,San Jose,California


Add geo coordinates to each city using geopy library

In [12]:
cityLoc = []
for i, val in enumerate(df_cities['City']):
    address = val + ', ' + df_cities.loc[i,'State[c]']
    geolocator = Nominatim(user_agent="city_explorer")
    location = geolocator.geocode(address)
    lat = location.latitude
    lng = location.longitude
    cityLoc.append([val, lat, lng])
cityLoc = pd.DataFrame(cityLoc)
cityLoc.columns = ['City', 'cityLat', 'cityLng']
cityLoc.head()

,City,cityLat,cityLng
0,New York,40.712728,-74.006015
1,Los Angeles,34.053691,-118.242767
2,Chicago,41.875562,-87.624421
3,Houston,29.758938,-95.367697
4,Phoenix,33.448437,-112.074142


Next I needed to know what type of venues are located in the selected cities in order to cluster them into groups.<br>
I used Foursquare’s ‘explore’ API call, however I faced an issue in retrieving reliable data.<br>
I needed to get venue data of 3,000 meter radius of the subject location, but if I simply set 3,000 meter radius and made 1 API call per city, I would get randomly selected 100 venues of the 3,000 meter radius area, because Foursquare's 'explore' function allows up to 100 calls at a time with my sandbox account.<br>
In most of the previously selected 20 populous cities, there are well more than 100 venues within 3,000 meter radius, so the result from this call was not reliable enough to reflect true structure of the city.<br>
Also, the result did not retrieve enough number of coffee shop data which will be used at the final screening stage.<br>
For these reasons, I divided each city into 9 areas and requested the results of 1,000 meter radius of the city's main location as well as its 8 surrounding areas with ±0.02 degrees on latitude and ±0.025 degrees on longitude (i.e. North, East, South, West, NE, SE, SW and NW), making API call 9 times separately for each city (see below map images).


---Note---<br>
Folium map might not be rendered on Github.<br>
If you can't see a map in below cell, please copy URL of this notebook and open it on nbviewer:
<a href="https://nbviewer.jupyter.org" target="_blank">https://nbviewer.jupyter.org</a>

In [13]:
print('3,000 meter radius of 1 location = 1 API call per city')
radLoc = [cityLoc.iloc[15, 1], cityLoc.iloc[15, 2]]
threekrad = folium.Map(location=radLoc, zoom_start=12, width=800, height=500)
folium.Circle(location=radLoc, radius=3000, fill_opacity=0.5).add_to(threekrad)
threekrad

3,000 meter radius of 1 location = 1 API call per city


In [47]:
print('1,000 meter radius of 9 areas = 9 API calls per city')
onekrad = folium.Map(location=radLoc, zoom_start=12, width=800, height=500)
folium.Circle(location=[radLoc[0]+0.02, radLoc[1]+0.025], radius=1000, fill_opacity=0.5).add_to(onekrad)
folium.Circle(location=[radLoc[0]+0.02, radLoc[1]-0.025], radius=1000, fill_opacity=0.5).add_to(onekrad)
folium.Circle(location=[radLoc[0]+0.02, radLoc[1]], radius=1000, fill_opacity=0.5).add_to(onekrad)
folium.Circle(location=[radLoc[0]-0.02, radLoc[1]+0.025], radius=1000, fill_opacity=0.5).add_to(onekrad)
folium.Circle(location=[radLoc[0]-0.02, radLoc[1]-0.025], radius=1000, fill_opacity=0.5).add_to(onekrad)
folium.Circle(location=[radLoc[0]-0.02, radLoc[1]], radius=1000, fill_opacity=0.5).add_to(onekrad)
folium.Circle(location=[radLoc[0], radLoc[1]+0.025], radius=1000, fill_opacity=0.5).add_to(onekrad)
folium.Circle(location=[radLoc[0], radLoc[1]-0.025], radius=1000, fill_opacity=0.5).add_to(onekrad)
folium.Circle(location=[radLoc[0], radLoc[1]], radius=1000, fill_opacity=0.5).add_to(onekrad)
onekrad

1,000 meter radius of 9 areas = 9 API calls per city


Now add 8 additional geo coordiantes of surrounding areas to the dataframe.<br>
There will be additional 8 for each of 20 cities, so total number of rows will be 20+(8*20)=180 rows.

In [15]:
surArea=[]
cityLoc_merged = cityLoc
for city, lat, lng in zip(cityLoc['City'], cityLoc['cityLat'], cityLoc['cityLng']):
    add = {'City':[city,city,city,city,city,city,city,city],
           'cityLat':[lat, lat, lat+0.02, lat+0.02, lat+0.02, lat-0.02, lat-0.02, lat-0.02],
           'cityLng':[lng+0.025, lng-0.025, lng+0.025, lng, lng-0.025, lng+0.025, lng, lng-0.025]}
    surArea.append(add)
for i in range(len(surArea)):
    df_sur = pd.DataFrame(surArea[i])
    cityLoc_merged = pd.concat([cityLoc_merged, df_sur], ignore_index=True)
cityLoc_merged.shape

(180, 3)

Call Foursquare API and create a dataframe with a list of the venues.

In [16]:
CLIENT_ID = 'KRPQRX5MZF3U0KYGNULSSKE3XPMDE1RZXCW01Y0S1B4LHAY5'
CLIENT_SECRET = 'PF35NBSBPCWBST1QEGIPJV0S4CXZR0DM0ALQ0YE0ZITQWZ3F'
VERSION = '20200707'

In [17]:
LIMIT = 100
radius = 1000

def getNearbyVenues(cityNames, cityLat, cityLng):
    venues_list=[]
    for cname, clat, clng in zip(cityNames, cityLat, cityLng):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, clat, clng, radius, LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(cname,
                             v['venue']['location']['lat'],
                             v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

In [19]:
venues = getNearbyVenues(cityNames=cityLoc_merged['City'],
                         cityLat=cityLoc_merged['cityLat'],
                         cityLng=cityLoc_merged['cityLng'])
venues.head()

,City,Venue Latitude,Venue Longitude,Venue Category
0,New York,40.711448,-74.006802,Hotel Bar
1,New York,40.711434,-74.006272,Laundry Service
2,New York,40.711173,-74.006702,Hotel
3,New York,40.713923,-74.005661,Dance Studio
4,New York,40.712415,-74.006724,Park


Next the 20 cities will be clustered.<br>
I used k-means method to group them into 5 clusters (i.e. k=5) based on the venue categories.<br>
As the venue category data were originally string type, one-hot encoding was applied to convert them into binary data.<br>
This binary data were then used to get the mean value of each venue category.<br>
This data describes each venue category’s frequency of occurrence and was used to fit with k-means.

Apply one-hot encoding to venue category.

In [20]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix='', prefix_sep='')
venues_onehot['City'] = venues['City']
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,City,ATM,Acai House,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Service,Alternative Healer,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,New York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,New York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,New York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,New York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,New York,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group by city and get frequency of each category

In [21]:
df_grouped_mean = venues_onehot.groupby('City').mean().reset_index()
df_grouped_mean.head()

,City,ATM,Acai House,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Service,Alternative Healer,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Austin,0.000000,0.0,0.000000,0.001477,0.0,0.001477,0.0,0.0,0.0,...,0.0,0.001477,0.005908,0.000000,0.000000,0.001477,0.001477,0.014771,0.0,0.0
1,Charlotte,0.000000,0.0,0.000000,0.002591,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.010363,0.005181,0.000000,0.000000,0.002591,0.007772,0.0,0.0
2,Chicago,0.000000,0.0,0.001538,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.001538,0.000000,0.000000,0.001538,0.003077,0.006154,0.0,0.0
3,Columbus,0.002268,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.004535,0.006803,0.004535,0.002268,0.004535,0.000000,0.011338,0.0,0.0
4,Dallas,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.000000,0.006579,0.000000,0.000000,0.004386,0.000000,0.004386,0.0,0.0


Cluster cities into 5 clusters using k-means method.

In [37]:
kclusters = 5
clustered = df_grouped_mean.drop('City', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(clustered)
kmeans.labels_[0:10]

array([1, 0, 2, 0, 0, 0, 0, 0, 0, 3], dtype=int32)

Add clustering label to original dataframe and see which cluster Seattle is in.

In [39]:
cityLoc.insert(0, 'Cluster Labels', kmeans.labels_)
cityLoc[cityLoc['City']=='Seattle']

,Cluster Labels,City,cityLat,cityLng
17,1,Seattle,47.603832,-122.330062


Here is a map with each city colored by cluster.<br>
---Note---<br>
Folium map might not be rendered on Github.<br>
If you can't see a map in below cell, please copy URL of this notebook and open it on nbviewer:
<a href="https://nbviewer.jupyter.org" target="_blank">https://nbviewer.jupyter.org</a>

In [40]:
usaCen = geolocator.geocode('USA')
map_clusters = folium.Map(location=[usaCen.latitude, usaCen.longitude], zoom_start=4, width=800, height=500)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []

for city, lat, lon, cluster in zip(cityLoc['City'], cityLoc['cityLat'], cityLoc['cityLng'], cityLoc['Cluster Labels']):
    label = folium.Popup(str(city) + ': Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True,
                        fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

Filter the cities with same cluster as Seattle, which is cluster 1.<br>
I found there are 5 out of the 20 cities previously screened.

In [41]:
similarCities = cityLoc[(cityLoc['Cluster Labels']==1) & (cityLoc['City'] !='Seattle')]
similarCities

,Cluster Labels,City,cityLat,cityLng
0,1,New York,40.712728,-74.006015
10,1,Austin,30.271129,-97.743700
13,1,Columbus,39.962260,-83.000707
15,1,San Francisco,37.779026,-122.419906
18,1,Denver,39.739236,-104.984862


So far I have screened our candidate cities to 5.<br>
Now I further screen them to final 3 candidates, based on the number of coffee shops in the city.

Get the number of venues in each category

In [42]:
df_grouped_count = venues_onehot.groupby('City').sum().reset_index()
df_grouped_count.head()

,City,ATM,Acai House,Accessories Store,Adult Boutique,Advertising Agency,African Restaurant,Airport,Airport Service,Alternative Healer,...,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Austin,0,0,0,1,0,1,0,0,0,...,0,1,4,0,0,1,1,10,0,0
1,Charlotte,0,0,0,1,0,0,0,0,0,...,0,0,4,2,0,0,1,3,0,0
2,Chicago,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,1,2,4,0,0
3,Columbus,1,0,0,0,0,0,0,0,0,...,0,2,3,2,1,2,0,5,0,0
4,Dallas,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,2,0,2,0,0


See how many coffee shops are in each city.<br>
I found that there are 3 categories that can all be considered coffee shops: 'Coffee Shop', 'Café', and 'Cafeteria'.<br>
I assumed them all as the same category, so I just added up their numbers and used the total.<br>
The column name will be 'Coffee Businesses'

In [43]:
coffee = pd.DataFrame(df_grouped_count['Café']+df_grouped_count['Coffee Shop']+df_grouped_count['Cafeteria'])
coffee['City'] = df_grouped_count['City']
coffee.rename(columns={0:'Coffee Businesses'}, inplace=True)
coffee.head()

,Coffee Businesses,City
0,38,Austin
1,21,Charlotte
2,35,Chicago
3,34,Columbus
4,28,Dallas


Lastly, get the number of coffe businesses for previously selected 5 candidate cities.

In [44]:
candidates = similarCities.join(coffee.set_index('City'), on='City')
candidates

,Cluster Labels,City,cityLat,cityLng,Coffee Businesses
0,1,New York,40.712728,-74.006015,56
10,1,Austin,30.271129,-97.743700,38
13,1,Columbus,39.962260,-83.000707,34
15,1,San Francisco,37.779026,-122.419906,71
18,1,Denver,39.739236,-104.984862,54


By sorting this list in descending order by the number of coffee businesses, final 3 candidate cities will be determined.

## Results
I reached 3 candidate cities that meet all 3 criteria.<br>
This is the final results and these 3 cities are recommended to the supplier as the candidate cities for the next target  market.

In [45]:
final_candidates = candidates.sort_values(by='Coffee Businesses', ascending=False).reset_index(drop=True).head(3)
final_candidates

,Cluster Labels,City,cityLat,cityLng,Coffee Businesses
0,1,San Francisco,37.779026,-122.419906,71
1,1,New York,40.712728,-74.006015,56
2,1,Denver,39.739236,-104.984862,54


## Recommendation & Discussion

With the results, I recommend with confidence following cities as the final candidates for the next target market:<br><br>
**- San Francisco (71 target coffee shops)**<br>
**- New York (56 target coffee shops)**<br>
**- Denver (54 target coffee shops)**<br><br>
The reasoning of this recommendation is based on solid data analysis, which demonstrated:
- These candidate cities are all highly populated.<br>
This factor works in favour of operating a business, in terms of stable revenue of target coffee shops and also attracting competitive human resource.
- Their urban structure is similar to Seattle.<br>
Since the supplier was already successful in Seattle market, this analysis took into account the success factors of the city, such as attractiveness for new residents and tourists.
- They have sufficient number of coffee shops to serve.<br>
The number of target coffee shops will have an impact on the supplier's future revenue, hence final candidates were chosen from top 3.

Although we chose 3 recommended cities, San Francisco is highly recommended among them in terms of the number of target coffee shops and its location within the same time zone of the supplier's current office in Seattle.<br>
If they rather prefer highly populated city, they might choose New York as it is ranked higher than San Francisco on the populous cities list.

# Conclusion


In this report, I analysed which cities in US are attractive markets for Seattle-based coffee making supplier.<br>
The analysis is entirely objective and data-driven, based on external data including Foursquare's Places Data and Wikipedia.<br>
My final recommendations are San Francisco, New York, and Denver, all of which meet criteria for future success: highly populated, similar city to Seattle, and having a number of coffee shops to serve.<br>
Taking the supplier's preference into consideration, such as the number of coffee shops and population density, they are now ready to choose the next market.